In [19]:
from CoolProp.CoolProp import PropsSI
# use the letters:
# ... T for (T)emperature 
# ... P for (P)ressure
# ... D for (D)ensity (use the density to calculate the specific volume)
# ... Q for vapor quality x

In [122]:
#Boundary conditions

#Ambiant air
p0 = 1.03e5 #Pa        TBD
T0 = 273+12 #K         TBD

#Miscellaneous
P = 5e6 #W             TBD
R = 8.3145 #J / mol K  
Mm = 28.97e-3 #kg/mol
cv = 718 #J/ kg K      TBD
dt =1  #s
p_loss=0.01 #bar       TBD

#Cavern
V = 1000  #m^3          TBD
pmax = 80e5 #bar       TBD
m0 = p0*V/T0/R*Mm #kg

#Compressor
eff_c   = 0.9 #[-]
p_ratio_c = 2 #[-]

#Turbine
eff_t   = 0.9 #[-]
p_ratio_t = 2 #[-]


In [ ]:
#Functions

#Get temperature after (de)pressurising from original temp
def Tpres(T,p_ratio):
    return T*p_ratio**(R/(Mm*cv+R))

#Get enthalpy for known T,p
def hTp(p,T):
    return PropsSI("H","P",p,"T",T,"air")

#First approximation specific work using enthalpy
def W1(p_in,p_ratio,T_in):
    p_out = p_in*p_ratio       
    T_out = Tpres(T_in,p_ratio)
    h_in = hTp(p_in,T_in)
    h_out = hTp(p_in,T_out)
    return h_out-h_in

#Mass flow rate
def mdot1():
    Work1 = W1(p0,p_ratio,T0)
    Work2 = W1(p0*p_ratio,p_ratio,T0)
    m=P*(Work1/eff_c+Work2/eff_c)**(-1)
    return m

#Second approximation specific work including velocities
def W2():
    #...
    return "null"

#Mass flow rate corrected for kinetic changes
def mdot2():
    #...
    return "null"


In [133]:
#Reset conditions
p_cavern = p0
m = m0
t=0

In [134]:
#Fill cavern until full or no more power (tmax is max time there is power)
def fill_cavern(tmax):
    global p_cavern
    global m
    global t
    
    while p_cavern < 0.99*pmax and t < tmax:
        #Determine mass in cavern
        m += mdot1()*dt
    
        #Determine pressure in cavern
        p_cavern=m*R/Mm*T0/V
    
        #Determine required pressure ratio
        p_ratio=(p_cavern/1e5)**(1/2) + p_loss
    
        t += dt
    if t < tmax:
        print('Not all this power can be stored, {0:.1f}% can not be used!'.format((1-t/tmax)*100))
        print("It took {0:.2f} hours to fill the cavern.".format(t/3600))
    else:
        print("No more power available.")
        
    print("Now the pressure is {0:.2f} bar".format(p_cavern/1e5))
    t=0
    return p_cavern 


    


In [135]:
#First fill for 1.5 hour
fill_cavern(3600*1.5)

No more power available.
Now the pressure is 40.41 bar


4041113.2999667814

In [136]:
#Second fill for 1 hour
fill_cavern(3600)

No more power available.
Now the pressure is 66.67 bar


6666522.166610609